# pQTL relevant data extraction using PySpark on UKB RAP
This notebook is built upon two notebooks from DNAnexus:
https://github.com/dnanexus/OpenBio/blob/master/UKB_notebooks/ukb-rap-pheno-basic.ipynb
https://github.com/dnanexus/UKB_RAP/blob/main/proteomics/0_extract_phenotype_protein_data.ipynb

### This notebook consists of three sections:
### Step 1: Extract clinical phenotype data from dataset['participant'] 
### Step 2: Extract Olink proteomics from dataset['olink_instance_0']
### Step 3: Extract operation records from dataset['hesin_oper']

In [1]:
# Import packages
# dxpy allows python to interact with the platform storage
# Note: This notebook is using spark since the size of the dataset we're extracting
# (i.e. the number of fields) is too large for a single node instance.
import dxpy
import dxdata

import pandas as pd
import subprocess
import glob
import os
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext


# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
# Need to adjust this buffer otherwise will get an error in toPandas() call
conf = pyspark.SparkConf().set("spark.kryoserializer.buffer.max", "1024")

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
sqlContext = SQLContext(sc)

/cluster/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
# Normal spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
# sc = pyspark.SparkContext()
# spark = pyspark.sql.SparkSession(sc)

In [3]:
dxdata.__version__

'0.41.0'

In [4]:
# silence warning
import warnings
warnings.filterwarnings('ignore')

# Re-enable warnings after your code if you want to see warnings again in subsequent cells
# warnings.filterwarnings('default')

In [5]:
# Set the option to None to display all rows, or set a specific number
# pd.set_option('display.max_rows', None)

# Reset display options to default
# pd.reset_option('display.max_rows')

In [6]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

## Access dataset

In [7]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

### Dataset 'entities' are virtual tables linked to one another.

The main entity is 'participant' and corresponds to most pheno fields. Additional entities correspond to linked health care data.
Entities starting with 'hesin' are for hospital records; entities starting with 'gp' are for GP records, etc.

In [8]:
dataset.entities

[<Entity "participant">,
 <Entity "covid19_result_england">,
 <Entity "covid19_result_scotland">,
 <Entity "covid19_result_wales">,
 <Entity "gp_clinical">,
 <Entity "gp_scripts">,
 <Entity "gp_registrations">,
 <Entity "hesin">,
 <Entity "hesin_diag">,
 <Entity "hesin_oper">,
 <Entity "hesin_critical">,
 <Entity "hesin_maternity">,
 <Entity "hesin_delivery">,
 <Entity "hesin_psych">,
 <Entity "death">,
 <Entity "death_cause">,
 <Entity "omop_death">,
 <Entity "omop_device_exposure">,
 <Entity "omop_note">,
 <Entity "omop_observation">,
 <Entity "omop_drug_exposure">,
 <Entity "omop_observation_period">,
 <Entity "omop_person">,
 <Entity "omop_procedure_occurrence">,
 <Entity "omop_specimen">,
 <Entity "omop_visit_detail">,
 <Entity "omop_visit_occurrence">,
 <Entity "omop_dose_era">,
 <Entity "omop_drug_era">,
 <Entity "omop_condition_era">,
 <Entity "omop_condition_occurrence">,
 <Entity "omop_measurement">,
 <Entity "olink_instance_0">,
 <Entity "olink_instance_2">,
 <Entity "olink_

### Step 1: Accessing the main 'participant' entity
The extraction code follows some examples

In [9]:
participant = dataset['participant']

#### Constructing field names, given UKB showcase field id, instance id, and array id

For the main participant data, the Platform uses field names with the following convention:

|Type of field|Syntax for field name|Example|
|:------------|---------------------|-------|
|Neither instanced nor arrayed|`p<FIELD-ID>`|`p31`|
|Instanced but not arrayed|`p<FIELD-ID>_i<INSTANCE-ID>`|`p40005_i0`|
|Arrayed but not instanced|`p<FIELD-ID>_a<ARRAY-ID>`|`p41262_a0`|
|Instanced and arrayed|`p<FIELD-ID>_i<INSTANCE-ID>_a<ARRAY-ID>`|`p93_i0_a0`|

Lastly, the participant id field itself (EID) is named `eid`

If you know exactly the field names you want to work with, put them in a string array (we will see later how to use that):

In [10]:
# example
# field_names = ['eid', 'p31', 'p21022', 'p40005_i0', 'p93_i0_a0']

#### Looking up fields, given UKB showcase field id

If you know the field id but you are not sure if it is instanced or arrayed, and want to grab all instances/arrays (if any), use these:

In [11]:
# Returns all field objects for a given UKB showcase field id

def fields_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given UKB showcase field id

def field_names_for_id(field_id):
    return [f.name for f in fields_for_id(field_id)]

##### Examples:

In [12]:
# Participant sex
field_names_for_id('31')

['p31']

In [13]:
# Age when attending assessment centre has multiple instances (visits) 
field_names_for_id('21003')

['p21003_i0', 'p21003_i1', 'p21003_i2', 'p21003_i3']

In [14]:
field_names_for_id('21001') # BMI

['p21001_i0', 'p21001_i1', 'p21001_i2', 'p21001_i3']

In [15]:
field_names_for_id('22009') # PC

['p22009_a1',
 'p22009_a2',
 'p22009_a3',
 'p22009_a4',
 'p22009_a5',
 'p22009_a6',
 'p22009_a7',
 'p22009_a8',
 'p22009_a9',
 'p22009_a10',
 'p22009_a11',
 'p22009_a12',
 'p22009_a13',
 'p22009_a14',
 'p22009_a15',
 'p22009_a16',
 'p22009_a17',
 'p22009_a18',
 'p22009_a19',
 'p22009_a20',
 'p22009_a21',
 'p22009_a22',
 'p22009_a23',
 'p22009_a24',
 'p22009_a25',
 'p22009_a26',
 'p22009_a27',
 'p22009_a28',
 'p22009_a29',
 'p22009_a30',
 'p22009_a31',
 'p22009_a32',
 'p22009_a33',
 'p22009_a34',
 'p22009_a35',
 'p22009_a36',
 'p22009_a37',
 'p22009_a38',
 'p22009_a39',
 'p22009_a40']

#### Looking up fields by title keyword

If you remember part of the field title, use these:

In [16]:
# Returns all field objects for a given title keyword

def fields_by_title_keyword(keyword):
    from distutils.version import LooseVersion
    fields = list(participant.find_fields(lambda f: keyword.lower() in f.title.lower()))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given title keyword

def field_names_by_title_keyword(keyword):
    return [f.name for f in fields_by_title_keyword(keyword)]

# Returns all field titles for a given title keyword

def field_titles_by_title_keyword(keyword):
    return [f.title for f in fields_by_title_keyword(keyword)]

# Furhter information: https://github.com/dnanexus/OpenBio/blob/master/UKB_notebooks/ukb-rap-pheno-basic.ipynb

### Grabbing fields into a Spark DataFrame

The `participant.retrieve_fields` function can be used to construct a Spark DataFrame of the given fields.

By default, this retrieves data as encoded by UK Biobank. For example, field p31 (participant sex) will be returned as an integer column with values of 0 and 1. To receive decoded values, supply the `coding_values='replace'` argument.

For more information, see [Tips for Retrieving Fields](https://dnanexus.gitbook.io/uk-biobank-rap/getting-started/working-with-ukb-data#tips-for-retrieving-fields) in the documentation.

## Extract clinical data 

In [26]:
field_names = ['eid', 
               'p31',  # sex
               'p21022',  # age at recruitment
               'p21001_i0',  # BMI
               'p54_i0',  # UK Biobank assessment centre
               'p53_i0',  # Date of attending assessment centre p53_i0
               'p3166_i0_a0', # time of blood collection
               'p22019', # Sex_chromosome_aneuploidy
               'p22000', # genotype batch
               'p22006',# genetic grouping, 
               'p30901_i0', # olink plateID
              ] \
                + field_names_for_id('22009') 

# 41270 = ICD10

• Further informatiaon on Date of first in-patient diagnosis can be found at https://biobank.ndph.ox.ac.uk/crystal/field.cgi?id=41280:
The corresponding ICD-10 diagnosis codes can be found in data-field Field 41270 and the two fields can be linked using the array structure.

• CAD definition and risk factors:
J Am Coll Cardiol. 2018 Oct 16;72(16):1883-1893. 
https://pubmed.ncbi.nlm.nih.gov/30309464/


## Grabbing fields into a Spark DataFrame
The participant.retrieve_fields function can be used to construct a Spark DataFrame of the given fields.

By default, this retrieves data as encoded by UK Biobank. For example, field p31 (participant sex) will be returned as an integer column with values of 0 and 1. To receive decoded values, supply the coding_values='replace' argument.

For more information, see Tips for Retrieving Fields in the documentation.

In [27]:
# Grabbing fields into a Spark DataFrame
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect())

In [28]:
# See the first five entries as a Spark DataFrame:
# df.show(5, truncate=False)

In [29]:
# See the first five entries as a Pandas DataFrame:
df.limit(5).toPandas()

,eid,p31,p21022,p21001_i0,p54_i0,p53_i0,p3166_i0_a0,p22019,p22000,p22006,...,p22009_a31,p22009_a32,p22009_a33,p22009_a34,p22009_a35,p22009_a36,p22009_a37,p22009_a38,p22009_a39,p22009_a40
0,2789877,1,56,22.6630,11004,2008-04-07,2008-04-07 11:29:25,NaN,4.0,1.0,...,-1.047680,-0.68373,0.89316,0.975021,-2.887280,-0.888835,-0.966800,0.01902,-0.902692,1.708810
1,3070454,1,61,25.3439,11009,2008-03-14,2008-03-14 11:21:35,NaN,28.0,1.0,...,-0.381002,2.19338,-2.32264,3.257110,-1.101520,0.027381,3.911290,1.82667,-0.034788,-0.750132
2,2733030,0,59,24.8491,11016,2009-04-08,2009-04-08 11:11:28,NaN,7.0,1.0,...,-2.269420,3.45342,1.81036,-2.710140,0.148229,-3.243800,0.778766,1.95113,4.714450,-1.682480
3,3039657,1,57,29.3577,10003,2006-03-21,2006-03-21 09:50:23,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4404117,0,49,34.1036,11014,2010-06-22,2010-06-22 18:38:14,NaN,55.0,1.0,...,-7.316860,1.28335,3.97141,1.290280,1.064820,6.634080,3.033570,-1.99974,5.446890,-4.351530


In [30]:
# if the above looks good, go ahead and convert the entire spark data frame to pandas data frame 
pdf = df.toPandas()

In [31]:
print(pdf.columns)

Index(['eid', 'p31', 'p21022', 'p21001_i0', 'p54_i0', 'p53_i0', 'p3166_i0_a0',
       'p22019', 'p22000', 'p22006', 'p30901_i0', 'p22009_a1', 'p22009_a2',
       'p22009_a3', 'p22009_a4', 'p22009_a5', 'p22009_a6', 'p22009_a7',
       'p22009_a8', 'p22009_a9', 'p22009_a10', 'p22009_a11', 'p22009_a12',
       'p22009_a13', 'p22009_a14', 'p22009_a15', 'p22009_a16', 'p22009_a17',
       'p22009_a18', 'p22009_a19', 'p22009_a20', 'p22009_a21', 'p22009_a22',
       'p22009_a23', 'p22009_a24', 'p22009_a25', 'p22009_a26', 'p22009_a27',
       'p22009_a28', 'p22009_a29', 'p22009_a30', 'p22009_a31', 'p22009_a32',
       'p22009_a33', 'p22009_a34', 'p22009_a35', 'p22009_a36', 'p22009_a37',
       'p22009_a38', 'p22009_a39', 'p22009_a40'],
      dtype='object')


In [32]:
# Set the option to None to display all rows, or set a specific number
# pd.set_option('display.max_rows', None)

# Your code to display the Series or DataFrame
# For example, if you have a Series named non_na_counts:
# pdf.count()

In [33]:
## Given the information above, re-select the column if needed
#field_names = ['p31', 'p21022', 'p54_i0', 'p41202', 'p53_i0', 'p40000_i0']  \
#    + field_names_for_id('41280') 
#df = participant.retrieve_fields(names=field_names, engine=dxdata.connect())

In [34]:
# Reset display options to default
# pd.reset_option('display.max_rows')

In [35]:
# Saving as TSV file
pdf.to_csv('clinical_data.tsv', sep='\t', index=False)

### Step 2: Extract Olink proteomics from dataset['olink_instance_0']

In [36]:
olink = dataset['olink_instance_0']
# olink.fields # to list all

In [37]:
temp_list = olink.fields
type(temp_list)

list

In [38]:
#def field_names_for_id(field_id):
#    return [f.name for f in fields_for_id(field_id)]
olink_all_field_names = [f.name for f in olink.fields]

In [39]:
#field_names = ['eid', 'col6a3'] # select like this if you're interested in only a few protein
olink_all_field_names[:5] # check

['eid', 'a1bg', 'aamdc', 'aarsd1', 'abca2']

In [40]:
dfo = olink.retrieve_fields(names=olink_all_field_names, engine=dxdata.connect())

In [41]:
# to check
# dfo.head(5)

### In case of extracting particular proteins only:
olink_field_names = ['eid', 'col6a3']
dfo = olink.retrieve_fields(names=olink_field_names, engine=dxdata.connect())

In [42]:
# See the first five entries as a Pandas DataFrame:
# dfo.limit(5).toPandas()

In [4]:
dfo.count() # check rows (53016 individual's data)

NameError: name 'dfo' is not defined

In [2]:
pdfo = dfo.toPandas()

NameError: name 'dfo' is not defined

In [3]:
pdfo.shape # (53016, 2924)

NameError: name 'pdfo' is not defined

In [ ]:
# Alternative approach
# pdfo = dfo.toPandas() is very memory intensive. So instead, we can do this sequentially (if needed).
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql.window import Window

# Total number of rows in the DataFrame
total_rows = dfo.count()

# Number of chunks
num_chunks = 10

# Calculate the number of rows per chunk. Adding 1 to ensure the last chunk includes all remaining rows
rows_per_chunk = (total_rows // num_chunks) + (total_rows % num_chunks > 0)

# Initialize an empty list to store each chunk's pandas DataFrame
chunks_list = []

# Create a column 'row_id' to help in filtering rows for each chunk
dfo_with_id = dfo.withColumn("row_id", row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)

for i in range(num_chunks):
    # Calculate start index for the current chunk
    start_index = i * rows_per_chunk
    
    # End index is not needed as we limit the number of rows fetched
    chunk_df = dfo_with_id.filter(dfo_with_id.row_id >= start_index).limit(rows_per_chunk)
    
    # Convert the chunk to a pandas DataFrame and append to the list
    chunk_pd_df = chunk_df.drop("row_id").toPandas()
    chunks_list.append(chunk_pd_df)

# Concatenate all chunks to form the final pandas DataFrame
pdf = pd.concat(chunks_list, ignore_index=True)

# Checking the shape of the final DataFrame
print(pdf.shape)

In [1]:
pdfo.iloc[:5, :5] # check

NameError: name 'pdfo' is not defined

In [ ]:
pdfo.to_csv('olink_data.tsv', sep='\t', index=False)

In [5]:
%%bash
#dx upload clinical_data.tsv --dest UKB:/data/04.COL6A3/
#dx upload olink_data.tsv --dest UKB:/data/04.COL6A3/
dx upload ukbrap_extract_clinical_olink_data.ipynb --dest UKB:/data/04.COL6A3/

ID                          file-GjFBgJjJBPpvv7B2JQ2yQkzg
Class                       file
Project                     project-Gf7Jk4QJBPpxxxG2x187pyJg
Folder                      /data/04.COL6A3
Name                        ukbrap_extract_clinical_olink_data.ipynb
State                       closing
Visibility                  visible
Types                       -
Properties                  -
Tags                        -
Outgoing links              -
Created                     Wed Apr 10 15:09:43 2024
Created by                  satoshi.yoshiji
 via the job                job-GjF9jx0JBPpbPqBx1Fy7zKqY
Last modified               Wed Apr 10 15:09:44 2024
Media type                  
archivalState               "live"
cloudAccount                "cloudaccount-dnanexus"


## Below are miscellaneous codes

### if you want to combine data frame, use pandas's merge 
combined = pd.merge(pdf, df_final, on = 'eid', how= 'left')

### Filtering spark dataframes

Spark dataframes can be filtered using the syntax: `df.filter(expression)`

The expression can be either :

* a string expression, built using SQL field names (e.g. `p31`) and SQL operators (e.g. `=`, `NOT`, `OR`, `AND`)
  * example: `'(p21022 >= 50) AND (p31 = 0)'`
  

* a Python expression, built using Python object fields (e.g. `df.p31`) and Python operators (e.g. `==`, `!`, `|`, `&`)
  * example: `(df.p21022 >= 50) & (df.p31 == 0)`

#### Example: Participants above 50 years old and female

df.count()

### Using SQL syntax
df.filter('(p21022 >= 50) AND (p31 = 0)').count()

### Using Python syntax
df.filter((df.p21022 >= 50) & (df.p31 == 0)).count()

### Getting more information about fields

Field objects (such as those returned by `fields_for_id` and `fields_by_title_keyword` above, or via the `participant[<field_name>]` syntax) provide additional information such as codings and units.

####  Working with codings

participant['p31'].coding

participant['p31'].coding.codes

def field_codes_by_keyword(field_name, keyword):
    return dict([(k,v) for (k,v) in participant[field_name].coding.codes.items() if keyword.lower() in v.lower()])

field_codes_by_keyword('p31', 'female')

field_codes_by_keyword('p41202', 'obesity')

### Additional field information

### Get link to UKB documentation page
participant['p31'].linkout

### Get field units
participant['p21022'].units

### Saving results

### Saving as CSV file
df.toPandas().to_csv('results.csv', index=False)

### Saving as TSV file
df.toPandas().to_csv('results.tsv', sep='\t', index=False)

### Saving as DTA file (Stata)
df.toPandas().astype(str).replace('None|NaN|nan', '.', regex=True).to_stata('results.dta')

#### Writing results back to the project

%%bash
dx upload results.tsv --dest / 